In [2]:
import csv
import re
import pandas as pd
import random
import copy
import nltk

NAME = "fleep"
GENDER = "M"
CLASS = "hunter"
RACE = "dwarf"

quests = pd.read_csv("quests.csv")
#print(list(quests.columns.values))
details = quests.Details
rawStr = ""
for d in details:
    rawStr += str(d)

In [3]:
def get_ran_word(cfdist,word,sampleNum):
    d = copy.copy(cfdist[word])
    words = [None] * sampleNum
    for i in range(sampleNum):
        words[i] = d.max();
        del(d[words[i]]);
    return random.choice(words)

def generate_ran_sentence(cfdist, word, num=15):
    for i in range(num):
        print(word + " ")
        #print word + " "
        word = get_ran_word(cfdist,word,3)
        
def format_simple(document):
    doc = document.lower()                                           #convert to lower case
    doc = re.sub(r'[\.\?\{\}\[\]\|\(\)!,:\'-;\$\"]<>','',doc)         #remove punctuations
    doc = re.sub(r'[\s]+',' ', doc)                                  #remove whitespace clumps
    doc =  doc.strip()                                               #remove trailing whitespace
    return doc

def replace_variables(document):
    doc = document.lower()
    if GENDER == "M":
        doc = re.sub(r'\$g(.*):(.*);',r'\g<1>',doc)
    else:
        doc = re.sub(r'\$g(.*):(.*);',r'\g<2>',doc)
    doc = re.sub(r'\$c',CLASS,doc)
    doc = re.sub(r'\$n',NAME,doc)
    doc = re.sub(r'\$r',RACE,doc)
    return doc
    
def format_simple_keep_punctuation(document):
    doc = document.lower()                                           #convert to lower case
    doc = re.sub(r'[\[\]\|<>-]','',doc)                                  #remove unneeded punctuations
    doc = re.sub(r'([\.\?\{\}\[\]\(\)!,:-;\$\"\'])',r' \g<1> ',doc)           #put spaces between tagged punctuations
    doc = re.sub(r'[\s]+',' ', doc)                                  #remove whitespace clumps
    doc =  doc.strip()                                               #remove trailing whitespace
    return doc

In [4]:
rawStr = replace_variables(rawStr)
rawStr = format_simple_keep_punctuation(rawStr)
#tokens = nltk.word_tokenize(rawStr)
tokens = rawStr.split(' ')
text = nltk.Text(tokens)

In [5]:
postag = nltk.pos_tag(text)
#tag_fd = nltk.FreqDist(tag for (word, tag) in postag)

In [10]:
#print(tag_fd.most_common())

In [9]:
from nltk import PCFG
from nltk.parse.generate import generate, demo_grammar
from nltk import CFG
from operator import itemgetter
from random import shuffle

nonterms = {".",",",r"PRP$",":","$","\'\'", "(", ")", "CC", "CD", "DT", "EX", "FW", "IN", r"WP$"}
nonpos = {"i", "\"", "\'"}
termvalues = {}
#test = {"NN","JJ","JJR","MD","RB","VB","RBS","RBR","UH", "VBP", "VBZ", "WDT", "CC", "NNS", "PRP", "VBN", "EX"}

limit = 30
postags = postag
shuffle(postags)
postags.sort(key=itemgetter(1))
my_grammar = """
  S -> KI NK
  S -> COLCD N
  S -> TR NT
  S -> TK
  COLCD -> COL CD
  TK -> RT TO AND TAL TO NNP
  TK -> TR TO AND TAL TO NNP
  NK -> CD NN AT NNP
  NK -> NNP THE NNP
  NT -> TO NNP
  NT -> TO NNP AND KI CD NN
  NT -> TO NNP AND COL CD NN
  NT -> TO NNP AND KI NNP THE NNP
  N -> JJ NN FR NNP
  N -> JJ NN AT NNP
  N -> JJ NN
  N -> NN
  COL -> 'Collect' | 'Gather' | 'Find'
  FR -> 'from'
  KI -> 'Kill' | 'Slay'
  TR -> 'Travel' | 'Journey'
  AT -> 'At' | 'Located In'
  THE -> 'The'
  OF -> 'Of'
  AND -> 'And'
  RT -> 'Return'
  TAL -> 'Talk'
"""

my_grammar += "  " + "\n  ".join([(r"CD -> '" + str(num) + r"'") for num in [1,5,10]])
for (a,b) in postags:
    if b not in nonterms and a not in nonpos and len(a)>1:
        if termvalues.has_key(b) == False:
            termvalues[b] = 1
            my_grammar += "\n  " + "\n  ".join([(b + r" -> '" + a + r"'")])
        elif termvalues.get(b) < limit:
            termvalues[b] += 1
            my_grammar += "\n  " + "\n  ".join([(b + r" -> '" + a + r"'")]) 
        
#for (a,b) in postags:
#    if b not in nonterms and a not in nonpos:
#        if a == "you'll" and b not in test:
#            print(b +": " + a + " - " + b[0])

grammar = CFG.fromstring(my_grammar)
#print(grammar.productions())

#file = open('generation.txt', 'w')
#for sentence in generate(grammar):
    #file.write(' '.join(sentence))
#file.close()
fulllist = []
for sentence in generate(grammar):
    fulllist.append(sentence)

ranlist = [None]*100
for i in range(0,100):
    ranlist[i] = random.choice(fulllist)

for sentence in ranlist:
    print(' '.join(sentence))

#print(' '.join(random.choice(ranlist)))

#print(grammar)
#grammar += "\n\tN -> 'cat'"
#print(grammar)

#print(dir(CFG))
#print("\n")
#print(grammar.productions)


Travel to #19 And Slay zangus The zapnabber
Journey to zul And Find 5 bet
Travel to zul And Slay maestra The zul
Journey to zangus And Slay xerash The zapnabber
Journey to maestra And Kill zul The zeth
Travel to zul And Kill zangus The malaka
Journey to kreldig And Gather 10 time
Collect 5 dear deathcharger At kagrosh
Travel to mah And Slay zeth The maestra
Travel to zul And Slay zul The malaka
Journey to zangus And Kill kreldig The zeth
Journey to zapnabber And Kill zul The zeth
Journey to maestra And Slay 5 bomb
Journey to zangus And Slay xoroth The xerash
Travel to yer And Kill zangus The mug
Journey to kagrosh And Kill malaka The #19
Journey to zeth And Slay zul The #19
Journey to zul And Slay kagrosh The zukk
Travel to #19 And Kill zul The zul
Travel to kreldig And Slay kagrosh The maestra
Journey to zangus And Slay zeth The xoroth
Travel to zapnabber And Slay zul The kaw
Journey to zeth And Collect 5 horde
Travel to zeth And Slay mug The maestra
Travel to zukk And Gather 1 bay
Tr